# Baseline performance
In this notebook, I analyze the results for the baseline HGS-VRPTW implementation by [1]. 

I ran the original baseline implementation for two different experiments:
1. 20K NoImprovement, timeLimit